# Processing of Job Search Life Cycle Data

This is notebook #1. It was created to demonstrate the approach of how to build out an end-to-end data science project. 
The goal is to showcase steps in a project including:
- data wrangling,
- simple data engineering,
- some basic EDA,
- an analysis,
- a visualization,
- handling secure access via a personal access token, and
- some simple ML Ops. 

The subject matter examines hypothetical data from a job search for a data scientist position.

Created Sept 18, 2025

In [1]:
# Import libraries
import pandas as pd
import os
from dotenv import load_dotenv  #,dotenv_values

## Imports

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get the PAT from the environment variables
pat_token = os.getenv("PAT_TOKEN")
# pat_token = os.getenv("PWD")

# Check if the token was loaded successfully
if pat_token:
    print("PAT token loaded successfully.")
    # Now you can use the pat_token in your code, for example:
    # api_url = "https://api.github.com/user"
    # headers = {"Authorization": f"token {pat_token}"}
    # response = requests.get(api_url, headers=headers)
else:
    print("Error: PAT_TOKEN environment variable not found.")

PAT token loaded successfully.


In [3]:
# Import data
# Working with a live copy of the original data
filename = ('~/Documents/JobSearch/Potential Job List 2025.xlsx')
pdf_original_data = pd.read_excel(filename, sheet_name = 'yr2025', 
                skiprows=0, nrows=151, 
                usecols=["Company","InterviewOrNot",
                          "ResumeNum","notes"])
print(pdf_original_data.shape)
pdf_original_data

(150, 4)


,Company,InterviewOrNot,ResumeNum,notes
0,Republic Services,interviewed,V07.1,Failed
1,USAA,NaN,V_08.1,Rejection notice
2,IMPACT Technology Recruiting,NaN,V_08.2,NaN
3,Early Warning Services,NaN,V_08.3,Rejection notice
4,Workiva,NaN,V_08.2,NaN
...,...,...,...,...
145,Reaching Beyond LLC,NaN,V11.1,NaN
146,Beacon Hill Staffing Group,NaN,V11.1,NaN
147,Resideo,NaN,V11.1,NaN
148,OpenVPN,NaN,V11.1,NaN


 ## Initial EDA on Raw Data

In [7]:
pdf_original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Company         150 non-null    object        
 1   Job Title       150 non-null    object        
 2   InterviewOrNot  17 non-null     object        
 3   Location        144 non-null    object        
 4   Date Applied    150 non-null    datetime64[ns]
 5   ResumeNum       149 non-null    object        
 6   notes           76 non-null     object        
dtypes: datetime64[ns](1), object(6)
memory usage: 8.3+ KB


In [8]:
pdf_original_data.describe()

,Date Applied
count,150
mean,2025-07-28 16:00:00
min,2025-04-17 00:00:00
25%,2025-07-11 00:00:00
50%,2025-07-29 00:00:00
75%,2025-08-13 18:00:00
max,2025-09-17 00:00:00


In [4]:
# Examine the unique values in the 'notes' column. Use these in functions below.
pdf_original_data['notes'].unique()

array(['Failed', 'Rejection notice', nan, 'Reject', 'Hiring freeze',
       'Pending', 'Wrong role/fit/WLB/pay'], dtype=object)

## Create a Function for Each Node in the Sankey Plot

In [5]:
#### Function to count the number of applications
def no_response(df):
    return df['notes'].isnull().sum()

# Function to count rejection notices
def rejection_notice(df):
    return df[df['notes']=='Rejection notice'].shape[0]

# Function to count calls with recruiter
def calls_with_recruiter(df):
    l_recruiter_calls = ["Wrong role/fit/WLB/pay", "Failed","Hiring freeze", "Reject", "Pending"]
    return df[df['notes'].isin(l_recruiter_calls)].shape[0]

# Function to count wrong fit
def wrong_fit(df):
    return df[df['notes']=='Wrong role/fit/WLB/pay'].shape[0]

# Function to count Tech screen
def tech_screen(df):
    l_tech_screen = ["Failed","Hiring freeze", "Reject", "Pending"]
    return df[df['notes'].isin(l_tech_screen)].shape[0]

# Function to count Hiring freeze
def hiring_freeze(df):
    return df[df['notes']=='Hiring freeze'].shape[0]

# Function to count Failed
def failed(df):
    return df[df['notes']=='Failed'].shape[0]

# Function to count On-Site/Final
def onsite_final(df):
    l_onsite_final = ["Reject", "Pending", "Offer"]
    return df[df['notes'].isin(l_onsite_final)].shape[0]

# Function to count Rejected
def rejected(df):
    return df[df['notes']=='Reject'].shape[0]

# Function to count Pending
def pending(df):
    return df[df['notes']=='Pending'].shape[0]

# Function to count Offer
def offer(df):
    return df[df['notes']=='Offer'].shape[0]

## Create Output Dataset

In [6]:
# The final output needs to be a .csv file

pdf_output = pd.DataFrame(columns=['source','target','value'])

pdf_output.loc[0,:] = ['Applications', 'No response', no_response(pdf_original_data)]
pdf_output.loc[1,:] = ['Applications', 'Rejection notice', rejection_notice(pdf_original_data)]
pdf_output.loc[2,:] = ['Applications', 'Recruiter call', calls_with_recruiter(pdf_original_data)]
pdf_output.loc[3,:] = ['Recruiter call', 'Wrong role/fit/WLB/pay', wrong_fit(pdf_original_data)]
pdf_output.loc[4,:] = ['Recruiter call', 'Tech screen', tech_screen(pdf_original_data)]
pdf_output.loc[5,:] = ['Tech screen', 'Hiring freeze', hiring_freeze(pdf_original_data)]
pdf_output.loc[6,:] = ['Tech screen', 'Failed', failed(pdf_original_data)]
pdf_output.loc[7,:] = ['Tech screen', 'On-site/Final', onsite_final(pdf_original_data)]
pdf_output.loc[8,:] = ['On-site/Final', 'Reject', rejected(pdf_original_data)]
pdf_output.loc[9,:] = ['On-site/Final', 'Offer', offer(pdf_original_data)]
pdf_output.loc[10,:] = ['On-site/Final', 'Pending', pending(pdf_original_data)]

pdf_output

,source,target,value
0,Applications,No response,73
1,Applications,Rejection notice,59
2,Applications,Recruiter call,18
3,Recruiter call,Wrong role/fit/WLB/pay,5
4,Recruiter call,Tech screen,13
5,Tech screen,Hiring freeze,3
6,Tech screen,Failed,2
7,Tech screen,On-site/Final,8
8,On-site/Final,Reject,4
9,On-site/Final,Offer,0


## Write Output Data to GitHub

In [7]:
# Programatically update csv file in GitHub repo with commit message
from github import Github
from datetime import datetime
import requests

repo_owner = 'rlucas49'
repo_name = 'EndToEnd_Project1_DataScienceJobSearch'
file_name = 'InputDataForSankey.csv'
token = pat_token
formatted_datetime = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
commit_message = f"Update CSV file at {formatted_datetime}"

github = Github(token)
repo = github.get_user(repo_owner).get_repo(repo_name)

url = f'https://raw.githubusercontent.com/{repo_owner}/{repo_name}/refs/heads/main/{file_name}'
response = requests.get(url)


# Convert DataFrame back to CSV string
csv_content = pdf_output.to_csv(index=False)

# Get file metadata (needed for sha)
contents = repo.get_contents(file_name, ref="main")

repo.update_file(path= file_name, 
                 message= commit_message, 
                 content= csv_content, 
                 sha= contents.sha, 
                 branch="main")

{'commit': Commit(sha="9547d264dea6777778aa2dfbd0df471b05edd674"),
 'content': ContentFile(path="InputDataForSankey.csv")}